
# <div style="text-align: center;">  Підготовка та аналіз даних  </div> #

# <div style="text-align: center;"> _Лабораторна робота №4, частина 1_ </div> #


## <div style="text-align: right;"> _ФБ-24 Бадрак Іван_ </div> ##





### 1. Розархівовуємо набір даних Individual household electric power consumption Data Set.
### Потім конвертуємо розархівований файл з .txt формату в .csv.
### Оброблюємо наш датафрейм, форматуємо дані та видаляємо пропущені значення.

In [1]:
import zipfile
import csv
import pandas as pd
from datetime import time as dt_time

file = open('household_power_consumption.csv', 'r')

def extract_zipfile():
    dir = "lab4"

    with zipfile.ZipFile('household_power_consumption.zip', 'r') as zip_ref:
        zip_ref.extractall(dir)
    print("Успішно розархівовано.")

def to_csv():
    txt_file = 'lab4/household_power_consumption.txt'
    csv_file = 'household_power_consumption.csv'

    with open(txt_file, 'r') as file_in, open(csv_file, 'w', newline='') as file_out:
        reader = csv.reader(file_in, delimiter=';')
        writer = csv.writer(file_out)

        for row in reader:
            writer.writerow(row)
    print(f"Успішно конвертовано {txt_file} у {csv_file}")

def pd_df():

    dtype = {
        'Global_active_power': float,
        'Global_reactive_power': float,
        'Voltage': float,
        'Global_intensity': float,
        'Sub_metering_1': float,
        'Sub_metering_2': float,
        'Sub_metering_3': float
    }

    na_values = ['?']
    df = pd.read_csv(file, dtype=dtype, na_values=na_values)

    df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)
    df['Time'] = pd.to_datetime(df['Time'], format='%H:%M:%S').dt.time
    df = df.drop(df.loc[(df['Sub_metering_1'] == 0) & (df['Sub_metering_2'] == 0) & (df['Sub_metering_3'] == 0)].index).dropna()
    return df

extract_zipfile()
to_csv()
df = pd_df()
# print (df)

Успішно розархівовано.
Успішно конвертовано lab4/household_power_consumption.txt у household_power_consumption.csv


### Далі виконуємо завдання 1-5. Спочатку використовуємо dataframe, потім numpy. Також порівнюємо часові витрати на виконання процедур.
##### 1. Обрати всі домогосподарства, у яких загальна активна споживана потужність перевищує 5 кВт.
##### 2. Обрати всі домогосподарства, у яких вольтаж перевищую 235 В.
##### 3. Обрати всі домогосподарства, у яких сила струму лежить в межах 19-20 А, для них виявити ті, у яких пральна машина та холодильних споживають більше, ніж бойлер та кондиціонер.
##### 4. Обрати випадковим чином 500000 домогосподарств (без повторів елементів вибірки), для них обчислити середні величини усіх 3-х груп споживання електричної енергії, а також
##### 5. Обрати ті домогосподарства, які після 18-00 споживають понад 6 кВт за хвилину в середньому, серед відібраних визначити ті, у яких основне споживання електроенергії у вказаний проміжок часу припадає на пральну машину, сушарку, холодильник та освітлення (група 2 є найбільшою), а потім обрати кожен третій результат із першої половини та кожен четвертий результат із другої половини.

In [2]:
from datetime import datetime
import numpy as np
import time

# 1.
def task1df(df):
    proc_df = df[df['Global_active_power'] > 5.0]
    return proc_df

# 2.
def task2df(df):
    proc_df = df[df['Voltage'] > 235.0]
    return proc_df

# 3. 
def task3df(df):
    proc_df = df[(df['Global_intensity'] <= 20.0) & (df['Global_intensity'] >= 19.0)]
    proc_df = proc_df[(proc_df['Sub_metering_2'] > proc_df['Sub_metering_3']) & (proc_df['Sub_metering_2'] > proc_df['Sub_metering_1'])]
    return proc_df

# 4. 
def task4df(df):
    proc_df = df.sample(500000)
    # print (proc_df)
    # print (len(proc_df))
    avg_cons_values = proc_df[['Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']].mean()
    return avg_cons_values

# 5. 
def task5df(df):
    proc_df = df[(df['Time'] > datetime.strptime('18:00:00', '%H:%M:%S').time()) & (df['Global_active_power'] > 6.0)]
    proc_df2 = proc_df[(proc_df['Sub_metering_2'] > proc_df['Sub_metering_1']) &
                              (proc_df['Sub_metering_2'] > proc_df['Sub_metering_3']) &
                              (proc_df['Sub_metering_2'] > proc_df['Sub_metering_2'].mean())]
    first_half = proc_df2.head(len(proc_df2) // 2)
    second_half = proc_df2.tail(len(proc_df2) // 2)
    aft_sampling = pd.concat([first_half.iloc[::3, :], second_half.iloc[::4, :]])
    return aft_sampling

# 1 numpy.
def task1numpy(df):
    proc_df = df[np.array(df['Global_active_power'] > 5.0)]
    return proc_df

# 2 numpy.
def task2numpy(df):
    proc_df = df[np.array(df['Voltage'] > 235)]
    return proc_df

# 3 numpy.
def task3numpy(df):
    proc_df = df[(df['Global_intensity'].values >= 19) & (df['Global_intensity'].values <= 20)]
    proc_df = proc_df[(proc_df['Sub_metering_2'].values > proc_df['Sub_metering_3'].values) &
                    (proc_df['Sub_metering_2'].values > proc_df['Sub_metering_1'].values)]
    return proc_df

# 4 numpy.
def task4numpy(df, n=500000):
    sampled_indices = np.random.choice(df.index, n, replace=True)
    proc_df = df.loc[sampled_indices]
    # print (len(proc_df))
    avg_cons_values = np.mean(proc_df[['Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']].values, axis=0)
    return avg_cons_values

# 5 numpy.
def task5numpy(df):
    proc_df = df[np.array((df['Time'] > dt_time(hour=18))) & (np.array(df['Global_active_power'] > 6.0))]
    proc_df = proc_df[np.array((proc_df['Sub_metering_2'] > proc_df['Sub_metering_1']) &
                              (proc_df['Sub_metering_2'] > proc_df['Sub_metering_3']) &
                              (proc_df['Sub_metering_2'] > proc_df['Sub_metering_2'].mean()))]
    first_half = proc_df.head(len(proc_df) // 2)
    second_half = proc_df.tail(len(proc_df) // 2)
    aft_sampling = pd.concat([first_half.iloc[::3, :], second_half.iloc[::4, :]])

    return aft_sampling


In [39]:
print("\nTask 1 DF")
start_time = time.time()
proc_df = task1df(df)
print(proc_df.head())
end_time = time.time() - start_time

print("\nTask 1 Numpy")
start_time_np = time.time()
proc_df = task1numpy(df)
print(proc_df.head())
end_time_np = time.time() - start_time_np
print(f"Час виконання:\nDF: {end_time} Numpy:{end_time_np} сек, DF швидше за Numpy у {end_time/end_time_np} разів\n ------------------------------------------ \n")


Task 1 DF
         Date      Time  Global_active_power  Global_reactive_power  Voltage  \
1  2006-12-16  17:25:00                5.360                  0.436   233.63   
2  2006-12-16  17:26:00                5.374                  0.498   233.29   
3  2006-12-16  17:27:00                5.388                  0.502   233.74   
11 2006-12-16  17:35:00                5.412                  0.470   232.78   
12 2006-12-16  17:36:00                5.224                  0.478   232.99   

    Global_intensity  Sub_metering_1  Sub_metering_2  Sub_metering_3  
1               23.0             0.0             1.0            16.0  
2               23.0             0.0             2.0            17.0  
3               23.0             0.0             1.0            17.0  
11              23.2             0.0             1.0            17.0  
12              22.4             0.0             1.0            16.0  

Task 1 Numpy
         Date      Time  Global_active_power  Global_reactive_power 

In [40]:
print("\nTask 2 DF")
start_time = time.time()
proc_df = task2df(df)
print(proc_df.head())
end_time = time.time() - start_time

print("\nTask 2 Numpy")
start_time_np = time.time()
proc_df = task2numpy(df)
print(proc_df.head())
end_time_np = time.time() - start_time_np
print(f"Час виконання:\nDF: {end_time} Numpy:{end_time_np} сек, DF швидше за Numpy у {end_time/end_time_np} разів\n ------------------------------------------ \n")


Task 2 DF
         Date      Time  Global_active_power  Global_reactive_power  Voltage  \
4  2006-12-16  17:28:00                3.666                  0.528   235.68   
5  2006-12-16  17:29:00                3.520                  0.522   235.02   
6  2006-12-16  17:30:00                3.702                  0.520   235.09   
7  2006-12-16  17:31:00                3.700                  0.520   235.22   
14 2006-12-16  17:38:00                4.054                  0.422   235.24   

    Global_intensity  Sub_metering_1  Sub_metering_2  Sub_metering_3  
4               15.8             0.0             1.0            17.0  
5               15.0             0.0             2.0            17.0  
6               15.8             0.0             1.0            17.0  
7               15.8             0.0             1.0            17.0  
14              17.6             0.0             1.0            17.0  

Task 2 Numpy
         Date      Time  Global_active_power  Global_reactive_power 

In [41]:
print("\nTask 3 DF")
start_time = time.time()
proc_df = task3df(df)
print(proc_df.head())
end_time = time.time() - start_time

print("\nTask 3 Numpy")
start_time_np = time.time()
proc_df = task3numpy(df)
print(proc_df.head())
end_time_np = time.time() - start_time_np
print(f"Час виконання:\nDF: {end_time} Numpy:{end_time_np} сек, DF швидше за Numpy у {end_time/end_time_np} разів\n ------------------------------------------ \n")


Task 3 DF
          Date      Time  Global_active_power  Global_reactive_power  Voltage  \
45  2006-12-16  18:09:00                4.464                  0.136   234.66   
460 2006-12-17  01:04:00                4.582                  0.258   238.08   
464 2006-12-17  01:08:00                4.618                  0.104   239.61   
475 2006-12-17  01:19:00                4.636                  0.140   237.37   
476 2006-12-17  01:20:00                4.634                  0.152   237.17   

     Global_intensity  Sub_metering_1  Sub_metering_2  Sub_metering_3  
45               19.0             0.0            37.0            16.0  
460              19.6             0.0            13.0             0.0  
464              19.6             0.0            27.0             0.0  
475              19.4             0.0            36.0             0.0  
476              19.4             0.0            35.0             0.0  

Task 3 Numpy
          Date      Time  Global_active_power  Global_re

In [42]:
print("\nTask 4 DF")
start_time = time.time()
proc_data = task4df(df)
print (proc_data)
end_time = time.time() - start_time

print("\nTask 4 Numpy")
start_time_np = time.time()
proc_data = task4numpy(df)
print (proc_data)
end_time_np = time.time() - start_time_np
print(f"Час виконання:\nDF: {end_time} Numpy:{end_time_np} сек, DF швидше за Numpy у {end_time/end_time_np} разів\n ------------------------------------------ \n")


Task 4 DF
Sub_metering_1    1.589862
Sub_metering_2    1.843306
Sub_metering_3    9.102638
dtype: float64

Task 4 Numpy
[1.572892 1.83491  9.118846]
Час виконання:
DF: 0.21365928649902344 Numpy:0.21633696556091309 сек, DF швидше за Numpy у 0.987622646666292 разів
 ------------------------------------------ 



In [43]:
print("\nTask 5 DF")
start_time = time.time()
proc_df = task5df(df)
print(proc_df.head())
end_time = time.time() - start_time

print("\nTask 5 Numpy")
start_time_np = time.time()
proc_df = task5numpy(df)
print(proc_df.head())
end_time_np = time.time() - start_time_np
print(f"Час виконання:\nDF: {end_time} Numpy:{end_time_np} сек, DF швидше за Numpy у {end_time/end_time_np} разів\n ------------------------------------------ \n")


Task 5 DF
            Date      Time  Global_active_power  Global_reactive_power  \
41    2006-12-16  18:05:00                6.052                  0.192   
44    2006-12-16  18:08:00                6.308                  0.116   
17496 2006-12-28  21:00:00                6.376                  0.252   
17499 2006-12-28  21:03:00                7.906                  0.286   
17502 2006-12-28  21:06:00                7.184                  0.000   

       Voltage  Global_intensity  Sub_metering_1  Sub_metering_2  \
41      232.93              26.2             0.0            37.0   
44      232.25              27.0             0.0            36.0   
17496   235.94              27.4             1.0            53.0   
17499   235.93              33.6             1.0            73.0   
17502   235.70              30.4             2.0            73.0   

       Sub_metering_3  
41               17.0  
44               17.0  
17496            17.0  
17499            17.0  
17502          

### У загальному випадку, Pandas та Numpy витрачали однакову кількість часу на виконання процедур, тому можна сказати, що немає сенсу віддавати перевагу тій чи іншій структурі даних в нашому випадку.